In [4]:
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from scipy.constants import h, k, c
from scipy.optimize import curve_fit

$ \text{Integrated Intensity} = \int I(v) \, dv $

$ T = \frac{h (\nu_2 - \nu_1)}{k \ln \left( \frac{I_1}{I_2} \right)} $

where $( \nu_1 )$ and $( \nu_2 )$ are the frequencies of the transitions, $( I_1 $) and $( I_2 $) are the integrated intensities, $( h $) is Planck's constant, and $( k $) is Boltzmann's constant.

In [23]:
'''
def read_fits_header(fits_file):
    """
    Read FITS header and extract relevant information.

    Parameters:
    - fits_file: Path to the FITS file

    Returns:
    - rest_frequency: Rest frequency in Hz
    - ra: Right Ascension (RA) in degrees
    - dec: Declination (Dec) in degrees
    - beam_area_sr: Estimated beam area in steradians (assuming circular beam)
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        
        rest_frequency = header['RESTFRQ']  # Rest frequency in Hz
        ra = header['OBSRA']                # RA in degrees
        dec = header['OBSDEC']              # Dec in degrees
        
        # Estimate beam area assuming a circular beam
        pixel_scale_ra = header['CDELT1'] * 3600.0  # RA pixel scale in arcseconds
        pixel_scale_dec = header['CDELT2'] * 3600.0  # Dec pixel scale in arcseconds
        beam_area_sr = np.pi * (pixel_scale_ra * pixel_scale_dec) / (4. * (180. / np.pi) ** 2)
        
    return rest_frequency, ra, dec, beam_area_sr
'''
#'''
def read_fits_header(fits_file):
    """
    Read FITS header and extract relevant information.

    Parameters:
    - fits_file: Path to the FITS file

    Returns:
    - rest_frequency: Rest frequency in Hz
    - ra: Right Ascension (RA) in degrees
    - dec: Declination (Dec) in degrees
    - beam_area_sr: Estimated beam area in steradians (assuming circular beam)
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        
        rest_frequency = header['RESTFRQ']  # Rest frequency in Hz
        ra = header['OBSRA']                # RA in degrees
        dec = header['OBSDEC']              # Dec in degrees
        
        # Check if bmin and bmaj are available in the header
        if 'BMIN' in header and 'BMAJ' in header:
            bmin = header['BMIN'] * (np.pi / 180.0) / 3600.0  # Convert from arcseconds to radians
            bmaj = header['BMAJ'] * (np.pi / 180.0) / 3600.0  # Convert from arcseconds to radians
            beam_area_sr = np.pi * bmin * bmaj 
        else:
            # Estimate beam area assuming a circular beam
            pixel_scale_ra = header['CDELT1'] * 3600.0  # RA pixel scale in arcseconds
            pixel_scale_dec = header['CDELT2'] * 3600.0  # Dec pixel scale in arcseconds
            beam_area_sr = np.pi * (pixel_scale_ra * pixel_scale_dec) / (4. * (180. / np.pi) ** 2)
        
    return rest_frequency, ra, dec, beam_area_sr
#'''

def integrate_intensity(intensity, velocity):
    """
    Integrate intensity over velocity.

    Parameters:
    - intensity: Array of intensity values (Jy/beam)
    - velocity: Array of velocity values (km/s)

    Returns:
    - Integrated intensity (Jy/beam * km/s)
    """
    print('intensity=',intensity)
    integrated_intensity = np.trapz(intensity, velocity)
    return integrated_intensity

def convert_jy_per_beam_to_w_per_m2_hz(intensity_jy, beam_area_sr):
    """
    Convert intensity from Jy/beam to W/m^2/Hz.

    Parameters:
    - intensity_jy: Intensity in Jy/beam
    - beam_area_sr: Beam area in steradians

    Returns:
    - Intensity in W/m^2/Hz
    """
    intensity_w_m2_hz = intensity_jy * 1e-26 / beam_area_sr
    return intensity_w_m2_hz

def calculate_excitation_temperature(intensity_1, intensity_2, nu_1, nu_2):
    """
    Calculate the excitation temperature.

    Parameters:
    - intensity_1: Integrated intensity of transition 1 in W/m^2/Hz
    - intensity_2: Integrated intensity of transition 2 in W/m^2/Hz
    - nu_1: Frequency of transition 1 in Hz
    - nu_2: Frequency of transition 2 in Hz

    Returns:
    - Excitation temperature in K
    """
    energy_1 = h * nu_1 / k
    energy_2 = h * nu_2 / k

    ratio = np.abs(intensity_2 / intensity_1)
    temperature = (energy_2 - energy_1) / np.log(ratio)

    return np.abs(temperature)

def extract_spectrum(fits_file, ra, dec):
    """
    Extracts the spectrum for a given RA and Dec from a 4D or 3D FITS cube.

    Parameters:
    - fits_file: Path to the FITS file
    - ra: Right Ascension in degrees
    - dec: Declination in degrees

    Returns:
    - Velocity array (km/s) and intensity array (Jy/beam)
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header)
        data = hdul[0].data
    
    # Check if the data has more than 3 dimensions
    if data.ndim > 3:
        # Extract the first 3 dimensions
        data = data[0, :, :, :]
        # Update WCS to only consider the first 3 dimensions
        wcs = WCS(header, naxis=3)
    
    # Get the pixel coordinates for the given RA and Dec
    x, y, _ = wcs.world_to_pixel_values(ra, dec, 0)
    
    # Extract the spectrum at the given RA and Dec
    spectrum = data[:, int(y), int(x)]

    # Get the velocity axis
    #velocity_axis = wcs.pixel_to_world(np.arange(data.shape[0]), 0, 0)[0].to_value('km/s')
    velocity_axis = wcs.wcs_pix2world(np.arange(data.shape[0]), 0, 0, 0)[0]

    return velocity_axis, spectrum

def excitation_temperature(fits_file_1, fits_file_2):
    """
    Calculate excitation temperature from two FITS files.

    Parameters:
    - fits_file_1: Path to FITS file for transition 1
    - fits_file_2: Path to FITS file for transition 2

    Returns:
    - Excitation temperature in K
    """
    # Read FITS headers for transition 1
    nu_1, ra_1, dec_1, beam_area_sr_1 = read_fits_header(fits_file_1)

    # Read FITS headers for transition 2
    nu_2, ra_2, dec_2, beam_area_sr_2 = read_fits_header(fits_file_2)
    print('nu_1=',nu_1, 'nu_2=',nu_2)
    print('ra_1=',ra_1, 'ra_2=',ra_2)
    print('dec_1=',dec_1, 'dec_2=',dec_2)
    #nu_1 = 89.1885247e9
    #nu_2 = 356.734223e9

    # Check if RA and Dec are the same for both transitions
    if ra_1 != ra_2 or dec_1 != dec_2:
        print("Warning: RA or Dec values are different between the two FITS files.")
    
    # Extract spectra
    velocity_1, intensity_1_jy_beam = extract_spectrum(fits_file_1, ra_1, dec_1)
    velocity_2, intensity_2_jy_beam = extract_spectrum(fits_file_2, ra_2, dec_2)

    # Integrate intensity over velocity
    integrated_intensity_1 = integrate_intensity(intensity_1_jy_beam, velocity_1)
    print('integrated_intensity_1_jy=',integrated_intensity_1)
    integrated_intensity_2 = integrate_intensity(intensity_2_jy_beam, velocity_2)
    print('integrated_intensity_2_jy=',integrated_intensity_2)
    
    # Convert from Jy/beam to W/m^2/Hz
    intensity_1_w_m2_hz = convert_jy_per_beam_to_w_per_m2_hz(integrated_intensity_1, beam_area_sr_1)
    print('intensity_1_w_m2_hz=',intensity_1_w_m2_hz)
    intensity_2_w_m2_hz = convert_jy_per_beam_to_w_per_m2_hz(integrated_intensity_2, beam_area_sr_2)
    print('intensity_2_w_m2_hz=',intensity_2_w_m2_hz)
    
    # Calculate the excitation temperature
    excitation_temperature = calculate_excitation_temperature(intensity_1_w_m2_hz, intensity_2_w_m2_hz, nu_1, nu_2)

    return excitation_temperature

In [24]:
fits_file_1 ='/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 14498-5856 HCO+/IRAS 14498-5856 HCO+.fits' # FITS file for transition 1
fits_file_2 = '/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 14498 HCO+(4-3)/IRAS 14498 HCO+(4-3)_spectral_axis_vel.fits' # FITS file for transition 2

excitation_temperature = excitation_temperature(fits_file_1, fits_file_2)
#print(f'Excitation Temperature: {excitation_temperature:.4f} K')
print(f'Excitation Temperature: {excitation_temperature} K')

nu_1= 89188526000.0 nu_2= 356734242000.0
ra_1= 223.4283750601 ra_2= 223.4285625
dec_1= -59.14902754425 dec_2= -59.14906277778
intensity= [-0.00687592 -0.00480481 -0.00150426 ... -0.00948869 -0.00632323
  0.00243673]
integrated_intensity_1_jy= -0.004408787004513202
intensity= [-0.00113355  0.01320491 -0.00623603 ...  0.00489006  0.00759535
  0.00685296]
integrated_intensity_2_jy= 0.0001358626838875959
intensity_1_w_m2_hz= 1.1517401869382758e-24
intensity_2_w_m2_hz= 2.471527500444519e-13
Excitation Temperature: 0.4921113763256782 K


In [22]:
fits_file_1 ='/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 16060-5146 HCO+/IRAS 16060-5146 HCO+.fits' # FITS file for transition 1
fits_file_2 = '/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 16060 HCO+(4-3)/IRAS 16060 HCO+(4-3)_spectral_axis_vel.fits' # FITS file for transition 2

excitation_temperature = excitation_temperature(fits_file_1, fits_file_2)
#print(f'Excitation Temperature: {excitation_temperature:.4f} K')
print(f'Excitation Temperature: {excitation_temperature} K')

nu_1= 89188526000.0 nu_2= 356734242000.0
ra_1= 242.4702083334 ra_2= 242.4706875
dec_1= -51.91519444446 dec_2= -51.91519444444
intensity= [-0.03450109 -0.04069877 -0.04566975 ...  0.02866     0.02419933
  0.03137914]
integrated_intensity_1_jy= 0.04304614338898732
intensity= [-0.00592259 -0.00609982 -0.00618587 ... -0.01377946  0.01396312
 -0.00991977]
integrated_intensity_2_jy= -0.0009348081333834149
intensity_1_w_m2_hz= 1.2904084587286083e-11
intensity_2_w_m2_hz= -2.978272378604023e-12
Excitation Temperature: 8.757444308319444 K


for HCO+ 4-3 transition rest freq. = 356.734223e9
HCO+ (1-0) = 89.1885247e9
CO(4-3)=345.7959899e9

## taking only intensity peak

In [33]:
'''
def read_fits_header(fits_file):
    """
    Read FITS header and extract relevant information.

    Parameters:
    - fits_file: Path to the FITS file

    Returns:
    - rest_frequency: Rest frequency in Hz
    - ra: Right Ascension (RA) in degrees
    - dec: Declination (Dec) in degrees
    - beam_area_sr: Estimated beam area in steradians (assuming circular beam)
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        
        rest_frequency = header['RESTFRQ']  # Rest frequency in Hz
        ra = header['OBSRA']                # RA in degrees
        dec = header['OBSDEC']              # Dec in degrees
        
        # Estimate beam area assuming a circular beam
        pixel_scale_ra = header['CDELT1'] * 3600.0  # RA pixel scale in arcseconds
        pixel_scale_dec = header['CDELT2'] * 3600.0  # Dec pixel scale in arcseconds
        beam_area_sr = np.pi * (pixel_scale_ra * pixel_scale_dec) / (4. * (180. / np.pi) ** 2)
        
    return rest_frequency, ra, dec, beam_area_sr
'''
#'''
def read_fits_header(fits_file):
    """
    Read FITS header and extract relevant information.

    Parameters:
    - fits_file: Path to the FITS file

    Returns:
    - rest_frequency: Rest frequency in Hz
    - ra: Right Ascension (RA) in degrees
    - dec: Declination (Dec) in degrees
    - beam_area_sr: Estimated beam area in steradians (assuming circular beam)
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        
        rest_frequency = header['RESTFRQ']  # Rest frequency in Hz
        ra = header['OBSRA']                # RA in degrees
        dec = header['OBSDEC']              # Dec in degrees
        
        # Check if bmin and bmaj are available in the header
        if 'BMIN' in header and 'BMAJ' in header:
            bmin = header['BMIN'] * (np.pi / 180.0) / 3600.0  # Convert from arcseconds to radians
            bmaj = header['BMAJ'] * (np.pi / 180.0) / 3600.0  # Convert from arcseconds to radians
            beam_area_sr = np.pi * bmin * bmaj
        else:
            # Estimate beam area assuming a circular beam
            pixel_scale_ra = header['CDELT1'] * 3600.0  # RA pixel scale in arcseconds
            pixel_scale_dec = header['CDELT2'] * 3600.0  # Dec pixel scale in arcseconds
            beam_area_sr = np.pi * (pixel_scale_ra * pixel_scale_dec) / (4. * (180. / np.pi) ** 2)
        
    return rest_frequency, ra, dec, beam_area_sr
#'''

def gaussian(x, amp, cen, wid):
    """
    Gaussian function.
    
    Parameters:
    - x: Input array
    - amp: Amplitude
    - cen: Center
    - wid: Width
    
    Returns:
    - Gaussian values
    """
    return amp * np.exp(-(x - cen) ** 2 / (2 * wid ** 2))

def integrate_intensity(intensity, velocity, v_lsr):
    """
    Integrate intensity over a specified range around the peak from a Gaussian fit.

    Parameters:
    - intensity: Array of intensity values (Jy/beam)
    - velocity: Array of velocity values (km/s)
    - v_lsr: Velocity of the Local Standard of Rest (km/s)

    Returns:
    - Integrated intensity (Jy/beam * km/s)
    """
    #print('velocity=',velocity)
    # Fit Gaussian to the intensity data
    popt, _ = curve_fit(gaussian, velocity, intensity, p0=[np.max(intensity), v_lsr, 2.5])
    amp, cen, wid = popt
    print(amp,cen,wid)
    fwhm = 2 * np.sqrt(2 * np.log(2)) * abs(wid)
    lower_bound = cen - fwhm / 2
    upper_bound = cen + fwhm / 2

    if lower_bound > upper_bound:
        lower_bound, upper_bound = upper_bound, lower_bound
    #print('velocity=',velocity)
    print('lower_bound=',lower_bound, 'upper_bound=',upper_bound)
    # Mask intensity and velocity within the bounds
    mask = (velocity >= lower_bound) & (velocity <= upper_bound)
    #print(mask)
    selected_intensity = intensity[mask]
    #print('selected_intensity=',selected_intensity)
    selected_velocity = velocity[mask]
    #print('selected_velocity=',selected_velocity)

    # Integrate intensity within the bounds
    integrated_intensity = np.trapz(selected_intensity, selected_velocity)
    
    return integrated_intensity


def convert_jy_per_beam_to_w_per_m2_hz(intensity_jy, beam_area_sr):
    """
    Convert intensity from Jy/beam to W/m^2/Hz.

    Parameters:
    - intensity_jy: Intensity in Jy/beam
    - beam_area_sr: Beam area in steradians

    Returns:
    - Intensity in W/m^2/Hz
    """
    intensity_w_m2_hz = intensity_jy * 1e-26 / beam_area_sr
    return intensity_w_m2_hz

def calculate_excitation_temperature(intensity_1, intensity_2, nu_1, nu_2):
    """
    Calculate the excitation temperature.

    Parameters:
    - intensity_1: Integrated intensity of transition 1 in W/m^2/Hz
    - intensity_2: Integrated intensity of transition 2 in W/m^2/Hz
    - nu_1: Frequency of transition 1 in Hz
    - nu_2: Frequency of transition 2 in Hz

    Returns:
    - Excitation temperature in K
    """
    energy_1 = h * nu_1 / k
    energy_2 = h * nu_2 / k

    ratio = np.abs(intensity_2 / intensity_1)
    temperature = (energy_2 - energy_1) / np.log(ratio)

    return np.abs(temperature)

def extract_spectrum(fits_file, ra, dec):
    """
    Extracts the spectrum for a given RA and Dec from a 4D or 3D FITS cube.

    Parameters:
    - fits_file: Path to the FITS file
    - ra: Right Ascension in degrees
    - dec: Declination in degrees

    Returns:
    - Velocity array (km/s) and intensity array (Jy/beam)
    """
    with fits.open(fits_file) as hdul:
        header = hdul[0].header
        wcs = WCS(header)
        data = hdul[0].data
        
    # Check if the data has more than 3 dimensions
    if data.ndim > 3:
        # Extract the first 3 dimensions
        data = data[0, :, :, :]
        
        # Update WCS to only consider the first 3 dimensions
        wcs = WCS(header, naxis=3)
    #print('datashape=', data.shape)
    # Get the pixel coordinates for the given RA and Dec
    x, y, _ = wcs.world_to_pixel_values(ra, dec, 0)
    
    # Extract the spectrum at the given RA and Dec
    spectrum = data[:, int(y), int(x)]

    # Get the velocity axis
    #velocity_axis = wcs.pixel_to_world(np.arange(data.shape[0]), 0, 0)[0].to_value('km/s')
    #print('Test:', wcs.wcs_pix2world(0, 0, 1, 0))
    velocity_axis = wcs.wcs_pix2world(0, 0, np.arange(data.shape[0]), 0)[2]/1000.
    #print('data shape=', data.shape[0])
    #print('vel=',velocity_axis)
    return velocity_axis, spectrum

def excitation_temperature(fits_file_1, fits_file_2, v_lsr):
    """
    Calculate excitation temperature from two FITS files.

    Parameters:
    - fits_file_1: Path to FITS file for transition 1
    - fits_file_2: Path to FITS file for transition 2

    Returns:
    - Excitation temperature in K
    """
    # Read FITS headers for transition 1
    nu_1, ra_1, dec_1, beam_area_sr_1 = read_fits_header(fits_file_1)

    # Read FITS headers for transition 2
    nu_2, ra_2, dec_2, beam_area_sr_2 = read_fits_header(fits_file_2)
    #print('nu_1=',nu_1, 'nu_2=',nu_2)
    #print('ra_1=',ra_1, 'ra_2=',ra_2)
    #print('dec_1=',dec_1, 'dec_2=',dec_2)
    #nu_1 = 89.1885247e9
    #nu_2 = 356.734223e9

    # Check if RA and Dec are the same for both transitions
    if ra_1 != ra_2 or dec_1 != dec_2:
        print("Warning: RA or Dec values are different between the two FITS files.")
    
    # Extract spectra
    velocity_1, intensity_1_jy_beam = extract_spectrum(fits_file_1, ra_1, dec_1)
    velocity_2, intensity_2_jy_beam = extract_spectrum(fits_file_2, ra_2, dec_2)

    # Integrate intensity over velocity
    integrated_intensity_1 = integrate_intensity(intensity_1_jy_beam, velocity_1, v_lsr)
    #print('integrated_intensity_1_jy=',integrated_intensity_1)
    integrated_intensity_2 = integrate_intensity(intensity_2_jy_beam, velocity_2, v_lsr)
    #print('integrated_intensity_2_jy=',integrated_intensity_2)
    
    # Convert from Jy/beam to W/m^2/Hz
    intensity_1_w_m2_hz = convert_jy_per_beam_to_w_per_m2_hz(integrated_intensity_1, beam_area_sr_1)
    #print('intensity_1_w_m2_hz=',intensity_1_w_m2_hz)
    intensity_2_w_m2_hz = convert_jy_per_beam_to_w_per_m2_hz(integrated_intensity_2, beam_area_sr_2)
    #print('intensity_2_w_m2_hz=',intensity_2_w_m2_hz)
    
    # Calculate the excitation temperature
    excitation_temperature = calculate_excitation_temperature(intensity_1_w_m2_hz, intensity_2_w_m2_hz, nu_1, nu_2)

    return excitation_temperature

In [34]:
fits_file_1 ='/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 14498-5856 HCO+/IRAS 14498-5856 HCO+.fits' # FITS file for transition 1
fits_file_2 = '/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 14498 HCO+(4-3)/IRAS 14498 HCO+(4-3)_spectral_axis_vel.fits' # FITS file for transition 2
v_lsr = -50.03

excitation_temperature = excitation_temperature(fits_file_1, fits_file_2, v_lsr)
#print(f'Excitation Temperature: {excitation_temperature:.4f} K')
print(f'Rotational Temperature: {excitation_temperature} K')

0.6318283334024506 -50.54214936278057 0.9880575491669796
lower_bound= -51.705498223991846 upper_bound= -49.378800501569295
0.7819948842584796 -50.7834697594526 0.6699628426105058
lower_bound= -51.57229072505517 upper_bound= -49.99464879385003
Rotational Temperature: 0.4393550668350351 K


In [31]:
fits_file_1 ='/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 16060-5146 HCO+/IRAS 16060-5146 HCO+.fits' # FITS file for transition 1
fits_file_2 = '/media/sanjana/One Touch/Toolkit/OutflowProject/ALMAData/IRAS 16060 HCO+(4-3)/IRAS 16060 HCO+(4-3)_spectral_axis_vel.fits' # FITS file for transition 2

v_lsr = -89.95

excitation_temperature = excitation_temperature(fits_file_1, fits_file_2, v_lsr)
#print(f'Excitation Temperature: {excitation_temperature:.4f} K')
print(f'Rotational Temperature: {excitation_temperature} K')

-1.3073629841433358 -91.4016796246858 2.261030197112115
lower_bound= -94.06383923997574 upper_bound= -88.73952000939586
-0.3672684620053377 -92.01585550266641 0.9533384822348225
lower_bound= -93.13832578649938 upper_bound= -90.89338521883344
Rotational Temperature: 77.66994577316429 K
